# iFest UNPAD Yaudahlah: Data Cleaning v2

Kurniadi Ahmad Wijaya<sup>1</sup><br>
Kaenova Mahendra Auditama<sup>2</sup><br>
Fendi Irfan Amorokhman<sup>3</sup><br>
Ananda Affan Fattahila<sup>4</sup><br>
<sup>1</sup><a href="mailto:kurniadiwijaya@student.telkomuniversity.ac.id">kurniadiwijaya@student.telkomuniversity.ac.id</a><br>
<sup>2</sup><a href="mailto:kaenova@student.telkomuniversity.ac.id">kaenova@student.telkomuniversity.ac.id</a><br>
<sup>3</sup><a href="mailto:fendiirfan@student.telkomuniversity.ac.id">fendiirfan@student.telkomuniversity.ac.id</a><br>
<sup>4</sup><a href="mailto:affanfattahila@student.telkomuniversity.ac.id">affanfattahila@student.telkomuniversity.ac.id</a><br>
Informatics Engineering, Telkom University, Indonesia<br>
2021

In [1]:
# Optional
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Sastrawi

     |████████████████████████████████| 209 kB 15.2 MB/s 


In [3]:
import pandas as pd
import re
import string
from tqdm import tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

class DataCleaning:
  # Initialization
  factory     = StemmerFactory()
  stemmer     = factory.create_stemmer()
  kamus_alay1 = pd.read_csv('https://raw.githubusercontent.com/fendiirfan/Kamus-Alay/main/Kamu-Alay.csv')
  kamus_alay1 = kamus_alay1.set_index('kataAlay')
  kamus_alay2 = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
  kamus_alay2 = kamus_alay2.filter(['slang', 'formal'], axis=1)
  kamus_alay2 = kamus_alay2.drop_duplicates(subset=['slang'], keep='first')
  kamus_alay2 = kamus_alay2.set_index('slang')
  stopword1   = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header = None)[0])
  custom_word = ['nya', 'tolong', 'iya', 'guna', 'kasih', 'buka', 'hari', 'sih', 'mohon', 'baru', 'ovo', 'dana', 'linkaja', 'saku', 'sakuku', 'aplikasi']

  @classmethod
  def CleanDataFrame(cls, df, col_name, jum_minimum=None, minimum_kata=0):

    final_list_clean = []
    final_list_kotor = []

    if jum_minimum == None: jum_minimum = len(df)
    if len(df) < jum_minimum: raise "Jumlah Data Yang Diinginkan melebihi Data yang Ada"
    i = 0
    current = 0
    
    while i < len(df):
      current_kalimat = df.loc[i][col_name]
      clean_kalimat = cls.__cleanSentence__(current_kalimat)
      if (len(clean_kalimat.split(' ')) > minimum_kata):
        final_list_clean.append(clean_kalimat)
        final_list_kotor.append(current_kalimat)
        current += 1
        if current % 10 == 0:
          print("Memproses {} data".format(current))

      if current == jum_minimum:
        break
      i += 1
    
    data = {
        'raw': final_list_kotor,
        'processed': final_list_clean
    }

    return pd.DataFrame(data)

  @classmethod
  def CleanSentence(cls, text):
    return cls.__cleanSentence__(text)

  @classmethod
  def __cleanSentence__(cls, text):
    '''
    Melakukan prapemrosesan pada suatu kalimat dengan menghilangkan formatting pada kalimat,
    menghilangkan stopword pada kalimat, mengganti kata alay yang sudah terdefinisikan, serta
    melakukan stemming kalimat tersebut.
    '''
    print(text)
    # #
    # Cleaning Formatted Text using Regex
    # #
    text = re.sub(r'http\S+', '', text)
    text = re.sub('(@\w+|#\w+)','',text)
    #will replace the html characters with " "
    text=re.sub('<.*?>', '', text)  
    #To remove the punctuations

    ## kuganti jadi gini biar pasti, kalau pakai cara yang dulu, banyak kata2 yang kegabung -kaenova
    temp_text = list(text)
    for i in range(len(temp_text)):
      if temp_text[i] in string.punctuation:
        temp_text[i] = " "
    text = ''.join(temp_text)
    ## sebelumnya kaya gini -kaenova
    # text = text.translate(str.maketrans(' ',' ',string.punctuation))

    #will consider only alphabets
    text = re.sub('[^a-zA-Z]',' ',text) 
    #will replace newline with space
    text = re.sub("\n"," ",text)
    #will convert to lower case
    text = text.lower()
    # will replace a word
    text = re.sub("(username|user|url|rt|xf|fx|xe|xa)\s|\s(user|url|rt|xf|fx|xe|xa)","",text)
    # will repalce repated char
    text = re.sub(r'(\w)(\1{2,})', r"\1", text)
    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)
    # will replace space more than one
    text = re.sub('(s{2,})',' ',text)
    # will join the words
    text=' '.join(text.split())

    text_split = text.split(' ')
    # #
    # Mengganti kata-kata yang tidak baku
    # aku gapakai try catch lagi, lebih simple malah ini
    # #
    for i in range(len(text_split)):
      if text_split[i] in cls.kamus_alay1.index:
        text_split[i] = cls.kamus_alay1.loc[text_split[i]]['kataBaik']
      elif text_split[i] in cls.kamus_alay2.index:
        text_split[i] = cls.kamus_alay2.loc[text_split[i]]['formal']
      else:
        pass

    # #
    # Stemming
    # #
    stemmed_text = cls.stemmer.stem(text)

    # #
    # Removing Stopwords and custom word
    # #
    temp_text_split = []
    for i in range(len(text_split)):
      if (text_split[i] not in cls.stopword1) and (text_split[i] not in cls.custom_word) and (type(text_split[i]) == str):
        temp_text_split.append(text_split[i])

    final_text = ' '.join(temp_text_split)
    
    return final_text

## Data Crawling

In [ ]:
#@title Multiple CSV from a Folder

import os
directory = '/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/rawV2' #@param {type:"string"}
out_directory = '/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/processedV4' #@param {type:"string"}

if directory[len(directory)-1] != '/':
  directory = directory + '/'

if out_directory[len(out_directory)-1] != '/':
  out_directory = out_directory + '/'

files = os.listdir(directory)

for file in files:
    if file[len(file)-4:] == '.csv':
      print(file)
      df = pd.read_csv(directory+file)
      df_temp = DataCleaning.CleanDataFrame(df, 'comments', None, 10)
      df_temp.to_csv(out_directory+file, index=False)

In [5]:
DataCleaning.CleanSentence('V 4.18.1 banyak masalah nya, aplikasi gampang hang, setiap kali memulai aplikasi akun sering keluar sendiri dilaman awal masukin nomor hp sama sandi terus, tab favorit yg sudah disimpan hilang semuamya!!. Mending gk usah di keluarin pembaharuan jika banyak masalah !!! Pembaruan itu untuk ngilangin masalah (bug) bukan munculin masalah (bug). Hah, padahal udah lama gunain link aja, jadi niat mau ganti ke dompet digital yg sejenis deh...')

V 4.18.1 banyak masalah nya, aplikasi gampang hang, setiap kali memulai aplikasi akun sering keluar sendiri dilaman awal masukin nomor hp sama sandi terus, tab favorit yg sudah disimpan hilang semuamya!!. Mending gk usah di keluarin pembaharuan jika banyak masalah !!! Pembaruan itu untuk ngilangin masalah (bug) bukan munculin masalah (bug). Hah, padahal udah lama gunain link aja, jadi niat mau ganti ke dompet digital yg sejenis deh...


'masalah gampang hang kali memulai akun keluar dilaman awal masukkan nomor handphone sandi terus tab favorit disimpan hilang semuamya lebih baik gak perlu mengeluarkan pembaharuan masalah pembaruan untuk menghilangkan masalah bug munculin masalah bug hah gunain link jadi niat ganti dompet digital sejenis deh'

## Data Train Sentiment General External

In [16]:
df_train = pd.read_csv('/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/training/combined_balance_ridlife_indonlu_affan.csv')
df_clean = DataCleaning.CleanDataFrame(df_train, 'tweet', None, 10)

In [17]:
df_clean

,raw,processed
0,Saya baru ganti HP dan baru download lagi saku...,ganti handphone download masuk akun register k...
1,Transaksi gagal tapi saldo terpotong sudah 2x ...,transaksi gagal saldo terpotong beginih tahu s...
2,Berkali kali nlp halo bca buat pengaduan salah...,berkali kali nlp halo baca pengaduan salah tf ...
3,"Ga suka ya sama aplikasi sakuku ini, saya sala...",suka salah pasword terus pencet registrasi ula...
4,Semua OK kecuali cara top up nya yang paling s...,ok kecuali cara top up susah wallet indonesia ...
...,...,...
779,PENIPUAN media. Bonus gk dpt. Donload dr hari ...,penipuan media bonus gak donload pertama launc...
780,Sebaiknya saat registrasi bisa memasukan nomor...,sebaiknya registrasi memasukan nomor handphone...
781,"Aplikasinya ga ada brand BCA sama sekali, teru...",aplikasinya brand baca terus waktu daftar scro...
782,Aplikasi bagus nih.... BCA memang TOP BGT... t...,bagus baca top banget tinggal menunggu bonusny...


In [18]:
df_clean_temp = df_clean.copy()
df_clean_temp

,raw,processed
0,Saya baru ganti HP dan baru download lagi saku...,ganti handphone download masuk akun register k...
1,Transaksi gagal tapi saldo terpotong sudah 2x ...,transaksi gagal saldo terpotong beginih tahu s...
2,Berkali kali nlp halo bca buat pengaduan salah...,berkali kali nlp halo baca pengaduan salah tf ...
3,"Ga suka ya sama aplikasi sakuku ini, saya sala...",suka salah pasword terus pencet registrasi ula...
4,Semua OK kecuali cara top up nya yang paling s...,ok kecuali cara top up susah wallet indonesia ...
...,...,...
779,PENIPUAN media. Bonus gk dpt. Donload dr hari ...,penipuan media bonus gak donload pertama launc...
780,Sebaiknya saat registrasi bisa memasukan nomor...,sebaiknya registrasi memasukan nomor handphone...
781,"Aplikasinya ga ada brand BCA sama sekali, teru...",aplikasinya brand baca terus waktu daftar scro...
782,Aplikasi bagus nih.... BCA memang TOP BGT... t...,bagus baca top banget tinggal menunggu bonusny...


In [19]:
df_train = pd.read_csv('/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/trainingEksternalV1/combined_balance_ridlife_indonlu_affan.csv')
df_train = df_train.drop('Unnamed: 0', axis=1)
df_train = df_train.rename(columns={'tweet':'raw'})
df_train

,raw,labels
0,wanita yang paling malang di dunia adalah wani...,-1
1,kecewa banget sama carefour yang memaksa harus...,-1
2,sapa nak layan perangai annoying aku ni nanti wei,-1
3,bani kampret doyan micin kalau pakde jokowi im...,-1
4,jokowi janji satu muharam hari santri . demi d...,-1
...,...,...
14207,di tembak mati saja atau diberi hukuman mati ....,-1
14208,percayalah akan bnyak isu isu yg akan di lempa...,-1
14209,"pertama kali makan di sini , suasana nya sepi ...",1
14210,pebenda mimpi tadi ni aku dah balik mesia past...,1


In [26]:
from sklearn.utils import shuffle

# final_df = pd.merge(df_train, df_clean_temp, how='inner', on='raw')
# final_df = final_df.drop_duplicates(['raw'])
# final_df = final_df.reset_index(drop=True)

# final_df_negative = final_df[final_df['labels'] == -1]
# final_df_positive = final_df[final_df['labels'] == 1]

# if len(final_df_negative) < len(final_df_positive):
#   final_df_positive = final_df_positive.head(len(final_df_negative))
# else:
#   final_df_negative = final_df_negative.head(len(final_df_positive))

# print(len(final_df_negative), len(final_df_positive))

# final_banget_df = pd.concat([final_df_negative, final_df_positive])
# final_banget_df = shuffle(final_banget_df)
# final_banget_df.to_csv("/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/trainingEksternalV1/train.csv", index=False)

,raw,processed
0,Saya baru ganti HP dan baru download lagi saku...,ganti handphone download masuk akun register k...
1,Transaksi gagal tapi saldo terpotong sudah 2x ...,transaksi gagal saldo terpotong beginih tahu s...
2,Berkali kali nlp halo bca buat pengaduan salah...,berkali kali nlp halo baca pengaduan salah tf ...
3,"Ga suka ya sama aplikasi sakuku ini, saya sala...",suka salah pasword terus pencet registrasi ula...
4,Semua OK kecuali cara top up nya yang paling s...,ok kecuali cara top up susah wallet indonesia ...
...,...,...
779,PENIPUAN media. Bonus gk dpt. Donload dr hari ...,penipuan media bonus gak donload pertama launc...
780,Sebaiknya saat registrasi bisa memasukan nomor...,sebaiknya registrasi memasukan nomor handphone...
781,"Aplikasinya ga ada brand BCA sama sekali, teru...",aplikasinya brand baca terus waktu daftar scro...
782,Aplikasi bagus nih.... BCA memang TOP BGT... t...,bagus baca top banget tinggal menunggu bonusny...
